In [1]:
import tensorflow as tf

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob

引入

In [ ]:
imgs_path=glob.glob('/content/drive/MyDrive/fruit/*/*.jpg')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1Read folder

In [ ]:
len(imgs_path)

1200

length

In [ ]:
img_p=imgs_path[2]
img_p

'/content/drive/MyDrive/fruit/FreshOrange/FreshOrange (65).jpg'

Read the image of one of the tags

In [ ]:
img_p.split("/")[6]

'FreshOrange (65).jpg'

read a tag

In [ ]:
all_labels_name=[img_p.split("/")[5] for img_p in imgs_path]

Get all tag names

In [ ]:
all_labels_name[:6]

['FreshOrange',
 'FreshOrange',
 'FreshOrange',
 'FreshOrange',
 'FreshOrange',
 'FreshOrange']

In [ ]:
label_name = np.unique(all_labels_name)

Returns the unique value of the label

In [ ]:
len(label_name)

6

In [ ]:
label_to_index=dict((name,i)for i,name in enumerate(label_name))

In [ ]:
label_to_index

{'FreshApple': 0,
 'FreshBanana': 1,
 'FreshOrange': 2,
 'RottenApple': 3,
 'RottenBanana': 4,
 'RottenOrange': 5}

Iteratively add tag serial numbers, using the dictionary method

In [ ]:
index_to_label = dict((v,k)for k,v in label_to_index.items())

In [ ]:
index_to_label

{0: 'FreshApple',
 1: 'FreshBanana',
 2: 'FreshOrange',
 3: 'RottenApple',
 4: 'RottenBanana',
 5: 'RottenOrange'}

反向映射

In [ ]:
all_labels=[label_to_index.get(name)for name in all_labels_name]

Image tag name is mapped to index

# 2Split test data and training data

In [ ]:
random_index = np.random.permutation(len(imgs_path))

In [ ]:
imgs_path=np.array(imgs_path)[random_index]
all_labels=np.array(all_labels)[random_index]

Out of order picture index, index

In [ ]:
i=int(len(imgs_path)*0.8)

In [ ]:
train_path=imgs_path[:i]
train_labels=all_labels[:i]
test_path=imgs_path[i:]
test_labels=all_labels[i:]

Slice, 80% do training data

In [ ]:
train_ds=tf.data.Dataset.from_tensor_slices((train_path,train_labels))
test_ds=tf.data.Dataset.from_tensor_slices((test_path,test_labels))

Create train dataset

In [ ]:
def load_img(path,label):
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image,channels=3)
  image = tf.image.resize(image,[256,256])
  image = tf.cast(image,tf.float32)
  image = image/255
  return image,label

In [ ]:
train_ds = train_ds.map(load_img)
test_ds = test_ds.map(load_img)

In [ ]:
train_ds

<_MapDataset element_spec=(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
BATCH_SIZE=32

In [ ]:
train_ds = train_ds.repeat().shuffle(300).batch(BATCH_SIZE)

In [ ]:
train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3),
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6)
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc']
              )

In [ ]:
train_count = len(train_path)
test_count = len(test_path)

In [ ]:
steps_per_epoch = train_count//BATCH_SIZE
validation_steps = test_count//BATCH_SIZE

In [ ]:
history = model.fit(train_ds,epochs=20,
      steps_per_epoch = steps_per_epoch,
      validation_data = test_ds,
      validation_steps=validation_steps)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.epoch, history.history.get('acc'), label='acc')
plt.plot(history.epoch, history.history.get('val_acc'), label='val_acc')
plt.legend()

In [ ]:
plt.plot(history.epoch, history.history.get('loss'), label='loss')
plt.plot(history.epoch, history.history.get('val_loss'), label='val_loss')
plt.legend()

In [ ]:
def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  image = tf.io.decode_jpeg(image,channels=3)
  image = tf.image.resize(image,[256,256])
  image = tf.cast(image,tf.float32)
  image = image/255.0
  return image

In [ ]:
test_img = '.jpg'
test_tensor = load_and_preprocess_image(test_img)
test_tensor = tf.expand_dims(test_tensor,axis=0)
pred = model.predict(test_tensor)

In [ ]:
index_to_label.get(np.argmax(pred))

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model/my_model') # path to the SavedModel directory
tflite_model = converter.convert()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quant = converter.convert()

In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

tflite_model_quant = converter.convert()

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
from PIL import Image

In [ ]:
tflite_model_file = '/content/drive/MyDrive/quantify_Full.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

# 获取输入和输出张量的详细信息
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
def load_and_preprocess_image(train_path):
    img = Image.open(train_path)  # 加载图像
    img = img.resize((256, 256))  # 调整图像大小
    img_array = np.array(img)  # 将图像转换为数组
    img_array = np.expand_dims(img_array, axis=0).astype(np.float32)  # 调整数组形状并转换数据类型
    return img_array

In [ ]:
accuracy_count = 0
total_count = 200  # 假设 X_test 是包含图像路径的列表

for i in range(total_count):
    test_image_path = train_path[i]  # 获取图像路径
    test_image = load_and_preprocess_image(test_image_path)  # 加载和预处理图像

    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_label = np.argmax(output_data)

    # 假设 Y_test 是对应的标签数组
    if predicted_label == train_labels[i]:
        accuracy_count += 1

accuracy = accuracy_count / total_count
print(f'Model accuracy on test data: {accuracy:.2f}')

Model accuracy on test data: 0.15


[train_path](https://)=imgs_path[:i]
train_labels=all_labels[:i]